In [41]:
from google.colab import files
uploaded = files.upload()

Saving sampleScheduling.xlsx to sampleScheduling.xlsx


In [42]:
import numpy as np
import pandas as pd

In [43]:
df = pd.read_excel('/content/sampleScheduling.xlsx')
df = df.fillna('nan')
df

,Task,Predeces,Time
0,A,nan,0.6
1,B,A,0.3
2,C,nan,0.8
3,D,nan,0.7
4,E,"C,D",0.4
5,F,"B,E",0.8
6,G,F,0.5
7,H,F,0.9
8,I,G,0.2
9,J,"H,I",0.1


In [50]:
import copy
import itertools
import numpy as np
import pandas as pd

class Scheduling:
  def __init__(self):
    self.__flag = False

  def __checkValidity(self):
    try:
      if (self.__df.empty):
        print ('Error: DataFrame must not be empty!')
    except:
      print ('Error: Must be a type of Pandas DataFrame!')
      self.__flag = True
      return

    if (len(self.__df) < 2):
      print ('Error: DataFrame must have atleast 3 rows!')
      self.__flag = True
    
    try:
      if ('Time' not in self.__df.columns):
        print ('Error: DataFrame must have "Time" column!')
        self.__flag = True
    except:
        print ('Error: Not a valid DataFrame - this may be a "Series" object!')
        self.__flag = True

  def sampleInputFormat(self):
    data = [['A', 'nan', 0.6],
    ['B', 'A', 0.3],
    ['C', 'nan', 0.8],
    ['D', 'nan', 0.7],
    ['E', 'C,D', 0.4],
    ['F', 'B,E', 0.8],
    ['G', 'F', 0.5],
    ['H', 'F', 0.9],
    ['I', 'G', 0.2],
    ['J', 'H,I', 0.1]]
    return pd.DataFrame(data, columns=['Task', 'Predeces', 'Time'])

  def __splitParents(self):
    dataList = np.array(self.__df).tolist()
    nanShedulesList = [data for data in dataList if (data[1] == 'nan')]
    nanShedulesList = list(itertools.permutations(nanShedulesList))
    nanShedulesList = [list(shedule) for shedule in nanShedulesList]

    shedulesList = []
    for data in nanShedulesList:
      data = copy.deepcopy(data)
      for child in dataList:
        if (child not in data):
          child = copy.deepcopy(child)
          parents = child[1].split(',')
          child[1] = parents
          data.append(child)
      shedulesList.append(data)
    
    return shedulesList


  def __checkParent(self, data, priorityList):
    isParentDone = True
    if (type (data[1]) == list):
      parentCount = 0
      for p in data[1]:
        for prio in priorityList:
          if (p == prio[0]):
            parentCount += 1
            break
      if (parentCount != len(data[1])):
        isParentDone = False
    return isParentDone

  def __check(self, data, priorityList):
    have = False
    for prio in priorityList:
      if (data[0] == prio[0]):
        have = True
    return have

  def __makePriorityLists(self, shedulesList):
    shedulesPriorityLists = []
    for data in shedulesList:
      data = copy.deepcopy(data)
      priorityList = []
      count = 1

      for data1 in data:
        if ((self.__check(data1, priorityList) != True) and self.__checkParent(data1, priorityList)):
          data1 = copy.deepcopy(data1)
          data1.append(count)
          priorityList.append(data1)
        
          max = data1[2]
          for data2 in data:
            if ((self.__check(data2, priorityList) != True) and self.__checkParent(data2, priorityList) and data1 != data2):
              data2 = copy.deepcopy(data2)

              if (max + data2[2] <= self.__cycleTime):
                max = max + data2[2]
                if (np.round(max,1) == self.__cycleTime):
                  data2.append(count)
                  priorityList.append(data2)
                  break
                elif ((max < self.__cycleTime)):
                  data2.append(count)
                  priorityList.append(data2)

          count += 1
      
      if (count-1 <= self.__minWorkStation):
        shedulesPriorityLists.append(priorityList)
    return shedulesPriorityLists

  def __getIdleTimes(self, shedulesPriorityLists):
    idleTimesAll = []
    totalIdleTimes = []

    for priority in shedulesPriorityLists:
      idleTimes = []
      tIdleTime = 0
      time = priority[0][2]
      for i in range(1, len(priority)):
        if (priority[i-1][3] == priority[i][3]):
          time += priority[i][2]
          if (i == len(priority)-1):
            idletime = np.round(self.__cycleTime - time, 1)
            idleTimes.append(idletime)
            tIdleTime += idletime
        else:
          idletime = np.round(self.__cycleTime - time, 1)
          idleTimes.append(idletime)
          tIdleTime += idletime

          time = priority[i][2]
          if (i == len(priority)-1):
            idletime = np.round(self.__cycleTime - time, 1)
            idleTimes.append(idletime)
            tIdleTime += idletime
      idleTimesAll.append(idleTimes)
      totalIdleTimes.append(np.round(tIdleTime,1))
    return totalIdleTimes

  def __minValue(self, arrayList):
    if (len(arrayList) > 0):
      minValue = arrayList[0]
      index = 0
      for i in range(1, len(arrayList)):
        if (arrayList[i] < minValue):
          minValue = arrayList[i]
          index = i
      return [index, minValue]

    return [-1, -1]

  def __setAtributes(self, df, produce, produceTime, cycleTime, minWorkStation):
    self.__df = df
    self.__checkValidity()
    if (self.__flag == True):
      return

    self.__produce = produce
    self.__produceTime = produceTime
    self.__totalTime = self.__df['Time'].mean()*10
    if (cycleTime != None):
      self.__cycleTime = cycleTime
    else:
      self.__cycleTime = np.round(self.__produceTime/self.__produce, 1)
    
    if (minWorkStation != None):
      self.__minWorkStation = minWorkStation
    else:
      self.__minWorkStation = int(np.ceil(self.__totalTime/self.__cycleTime))

  def getOptimumScheduleUsingCycleTimes(self, df, produce, produceTime, cycleTimes, minWorkStation=None):
    if (type (cycleTimes) != list):
      return print('Error: cycleTimes must be a list')
    if (len(cycleTimes) < 1):
      return print('Error: cycleTimes list must have atleast one item.')
    for item in cycleTimes:
      if ((type (item) == str) or (type (item) == list)):
        return print('Error: cycleTimes list must have all integer or float data type values.')
    
    idletimesList = []
    for cycleTime in cycleTimes:
      self.__setAtributes(df, produce, produceTime, cycleTime, minWorkStation)
      shedulesList = self.__splitParents()
      shedulesPriorityLists =  self.__makePriorityLists(shedulesList)
      totalIdleTimes = self.__getIdleTimes(shedulesPriorityLists)
      [index, value] = self.__minValue(totalIdleTimes)
      idletimesList.append([cycleTime, index, value, shedulesPriorityLists])
    

    filteredIdletimesList = [item for item in idletimesList if item[2] > -1]
    if (len(filteredIdletimesList) > 0):
      minIdletime = filteredIdletimesList[0][2]
      indexOfItem = 0
      for i, item in enumerate(filteredIdletimesList):
        if (item[2] < minIdletime):
          minIdletime = item[2]
          indexOfItem = i
      
      optimumCycleTime = filteredIdletimesList[indexOfItem][0]
      dataIndex = filteredIdletimesList[indexOfItem][1]
      data = filteredIdletimesList[indexOfItem][3][dataIndex]

      print (f'The optimum cycle time: {optimumCycleTime}')
      print (f'The optimum idle time: {minIdletime}')
      print ('The optimum Schedule is:')
      finalDf = pd.DataFrame(data, columns=['Task', 'Dependencies', 'Time', 'Priority'])
      return finalDf
    else:
      print('Sorry. No optimized Schedules found!')
      return []

    return [dataIndex, cycleTime, data]

  
  def getOptimumSchedule(self, df, produce, produceTime, cycleTime=None, minWorkStation=None):
    self.__setAtributes(df, produce, produceTime, cycleTime, minWorkStation)

    if (self.__flag == True): 
      print ('Error was happened when calling the Class "Sheduling". Check the error messages avobe!')
      return

    shedulesList = self.__splitParents()
    shedulesPriorityLists =  self.__makePriorityLists(shedulesList)
    totalIdleTimes = self.__getIdleTimes(shedulesPriorityLists)
    [index, value] = self.__minValue(totalIdleTimes)
    if (index != -1):
      print (f'The optimum idle time: {value}')
      print ('The optimum Schedule is:')
      finalDf = pd.DataFrame(shedulesPriorityLists[index], columns=['Task', 'Dependencies', 'Time', 'Priority'])
      return finalDf
    else:
      print('Sorry. No optimized Schedules found!')
      return []

In [51]:
produce = 300
produceTime = 460

scheduling = Scheduling()
scheduling.getOptimumScheduleUsingCycleTimes(df, produce, produceTime, cycleTimes=[1.1, 1.2, 1.5, 1.4, 1.3, 1.6])

The optimum cycle time: 1.4
The optimum idle time: 0.3
The optimum Schedule is:


,Task,Dependencies,Time,Priority
0,A,nan,0.6,1
1,C,nan,0.8,1
2,D,nan,0.7,2
3,B,[A],0.3,2
4,E,"[C, D]",0.4,2
5,F,"[B, E]",0.8,3
6,G,[F],0.5,3
7,H,[F],0.9,4
8,I,[G],0.2,4
9,J,"[H, I]",0.1,4


In [46]:
produce = 300
produceTime = 460

# scheduling = Scheduling()
scheduling.getOptimumSchedule(df, produce, produceTime)

The optimum idle time: 0.7
The optimum Schedule is:


,Task,Dependencies,Time,Priority
0,A,nan,0.6,1
1,C,nan,0.8,1
2,D,nan,0.7,2
3,B,[A],0.3,2
4,E,"[C, D]",0.4,2
5,F,"[B, E]",0.8,3
6,G,[F],0.5,3
7,I,[G],0.2,3
8,H,[F],0.9,4
9,J,"[H, I]",0.1,4


In [47]:
cycleTime = 1.4
# scheduling = Scheduling()
scheduling.getOptimumSchedule(df, produce, produceTime, cycleTime)

The optimum idle time: 0.3
The optimum Schedule is:


,Task,Dependencies,Time,Priority
0,A,nan,0.6,1
1,C,nan,0.8,1
2,D,nan,0.7,2
3,B,[A],0.3,2
4,E,"[C, D]",0.4,2
5,F,"[B, E]",0.8,3
6,G,[F],0.5,3
7,H,[F],0.9,4
8,I,[G],0.2,4
9,J,"[H, I]",0.1,4


In [48]:
cycleTime = 1.3
minWorkStation = 3
# scheduling = Scheduling()
scheduling.getOptimumSchedule(df, produce, produceTime, cycleTime, minWorkStation)

Sorry. No optimized Schedules found!


[]

In [49]:
# scheduling = Scheduling()
scheduling.getOptimumSchedule([], produce, produceTime, cycleTime, minWorkStation)
print ('----------------------------------')
scheduling.getOptimumSchedule(df['Predeces'], produce, produceTime, cycleTime, minWorkStation)

Error: Must be a type of Pandas DataFrame!
Error was happened when calling the Class "Sheduling". Check the error messages avobe!
----------------------------------
Error: Not a valid DataFrame - this may be a "Series" object!
Error was happened when calling the Class "Sheduling". Check the error messages avobe!
